In [1]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
# Load the iris dataset
iris = datasets.load_iris()

# Create our X and y data
X = iris.data
y = iris.target
y[:5]
X[:5]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2]])

In [3]:
# Split the data into 70% training data and 30% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
# Train the scaler, which standarizes all the features to have mean=0 and unit variance
sc = StandardScaler()
sc.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [5]:
# Apply the scaler to the X training data
X_train_std = sc.transform(X_train)

# Apply the SAME scaler to the X test data
X_test_std = sc.transform(X_test)

In [148]:
# Create a perceptron object with the parameters: 40 iterations (epochs) over the data, and a learning rate of 0.1
ppn = Perceptron( max_iter = 350,eta0=0.00001,tol=None, penalty='l2')

# Train the perceptron
ppn.fit(X_train_std, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1e-05,
           fit_intercept=True, max_iter=350, n_iter_no_change=5, n_jobs=None,
           penalty='l2', random_state=0, shuffle=True, tol=None,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [149]:
# Apply the trained perceptron on the X data to make predicts for the y test data
y_pred = ppn.predict(X_test_std)
# View the accuracy of the model, which is: 1 - (observations predicted wrong / total observations)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.96


In [145]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train_std, y_train)
y_pred = clf.predict(X_test_std)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.98


In [79]:
from keras.models import Sequential
from keras.layers import Dense, Activation
model = Sequential()
model.add(Dense(3, input_shape=(4,),activation='softmax'))
#model.add(Activation('softmax'))

In [150]:
from keras.optimizers import SGD

In [80]:

sgd = SGD(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [46]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform( y_train.reshape(-1, 1))

In [ ]:
y

In [37]:
X_train_std.shape

(105, 4)

In [81]:
model.fit(X_train_std,y,epochs=200, validation_split=0.1)

y_pred = model.predict_classes(X_test_std)

Epoch 1/200
3/3 [==============================] - 0s 56ms/step - loss: 1.4387 - accuracy: 0.1489 - val_loss: 1.3915 - val_accuracy: 0.0909
Epoch 2/200
3/3 [==============================] - 0s 7ms/step - loss: 1.3936 - accuracy: 0.1596 - val_loss: 1.3476 - val_accuracy: 0.0909
Epoch 3/200
3/3 [==============================] - 0s 8ms/step - loss: 1.3508 - accuracy: 0.1915 - val_loss: 1.3058 - val_accuracy: 0.0909
Epoch 4/200
3/3 [==============================] - 0s 7ms/step - loss: 1.3105 - accuracy: 0.2021 - val_loss: 1.2660 - val_accuracy: 0.1818
Epoch 5/200
3/3 [==============================] - 0s 7ms/step - loss: 1.2720 - accuracy: 0.2340 - val_loss: 1.2283 - val_accuracy: 0.1818
Epoch 6/200
3/3 [==============================] - 0s 8ms/step - loss: 1.2351 - accuracy: 0.2553 - val_loss: 1.1924 - val_accuracy: 0.1818
Epoch 7/200
3/3 [==============================] - 0s 7ms/step - loss: 1.2006 - accuracy: 0.2660 - val_loss: 1.1583 - val_accuracy: 0.2727
Epoch 8/200
3/3 [=========

In [82]:
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 0.91


In [57]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________
